In [4]:

# Federated Learning Model Update Verification

from pathlib import Path
import pandas as pd
import torch
import sys
import os
sys.path.append("..")

from flsim.storage import IPFSSim


def load_all_runs(runs_root: Path) -> pd.DataFrame:
    runs_root = runs_root if runs_root.name == "runs" else (runs_root / "runs")
    rows = []
    if not runs_root.exists():
        print("No runs folder found:", runs_root)
        return pd.DataFrame()
    for exp_dir in sorted(runs_root.glob("*")):
        if not exp_dir.is_dir():
            continue
        for ts_dir in sorted(exp_dir.glob("*")):
            csv_path = ts_dir / "fl_log.csv"
            if csv_path.exists():
                try:
                    df = pd.read_csv(csv_path)
                    df["exp"] = exp_dir.name
                    df["ts"] = ts_dir.name
                    rows.append(df)
                except Exception as e:
                    print("Failed to read", csv_path, e)
    if not rows:
        return pd.DataFrame()
    return pd.concat(rows, ignore_index=True)


BASE = Path("/Users/yuandouwang/Documents/projects/federated_lab/runs")  # TODO: change to your project path
df = load_all_runs(BASE)
print("Loaded rows:", len(df))
df.head(3)



Loaded rows: 170


,round,global_acc,manifest_cid,node_id,samples,loss,acc,claimed_acc,eval_acc,acc_diff,...,balance,stake_penalty,stake,reputation,malicious_detected,committee,is_malicious,strategy,exp,ts
0,0,0.7703,Qm00000022,0,7682,0.353403,0.914150,0.9141,0.6532,0.2609,...,74.5680,0.0,174.5680,19.5688,1,1,0,none,benign,20250813-223511
1,0,0.7703,Qm00000022,1,6725,0.203163,0.945145,0.9451,0.3488,0.5963,...,39.7479,0.0,139.7479,19.0325,1,1,0,none,benign,20250813-223511
2,0,0.7703,Qm00000022,2,1321,0.341979,0.912869,0.9129,0.3931,0.5198,...,26.0427,0.0,126.0427,18.4381,1,1,0,none,benign,20250813-223511


In [5]:
import os
import torch
import pandas as pd
from pathlib import Path

def load_all_global_models(runs_root: Path, model_class=None, device='cpu'):
    runs_root = runs_root if runs_root.name == "runs" else (runs_root / "runs")
    rows = []

    if not runs_root.exists():
        print("No runs folder found:", runs_root)
        return pd.DataFrame()

    for exp_dir in sorted(runs_root.glob("*")):
        if not exp_dir.is_dir():
            continue
        for ts_dir in sorted(exp_dir.glob("*")):
            models_dir = ts_dir / "models"
            if not models_dir.exists():
                continue

            print(f"Loading models from: {models_dir}")
            for fname in sorted(os.listdir(models_dir)):
                if fname.endswith('.pt') or fname.endswith('.pth'):
                    fpath = models_dir / fname
                    try:
                        state_dict = torch.load(fpath, map_location=device)
                        model = None
                        if model_class is not None:
                            model = model_class().to(device)
                            model.load_state_dict(state_dict, strict=False)
                            model.eval()

                        # Extract round number if in filename (e.g., "global_round_3.pt")
                        round_num = None
                        if "round" in fname:
                            try:
                                round_num = int("".join([c for c in fname if c.isdigit()]))
                            except ValueError:
                                pass

                        rows.append({
                            "exp": exp_dir.name,
                            "ts": ts_dir.name,
                            "round": round_num,
                            "fname": fname,
                            "state_dict": state_dict,
                            "model": model
                        })
                        print(f"✅ Loaded model {fname}")
                    except Exception as e:
                        print(f"❌ Failed to load {fpath}: {e}")

    return pd.DataFrame(rows)

# Example usage
BASE = Path("/Users/yuandouwang/Documents/projects/federated_lab/runs")  # Change to your project path
models_df = load_all_global_models(BASE)
print(f"Loaded {len(models_df)} models")
models_df.head(3)


Loading models from: /Users/yuandouwang/Documents/projects/federated_lab/runs/benign/20250813-223511/models
✅ Loaded model global_round_0.pt
✅ Loaded model global_round_1.pt
✅ Loaded model global_round_2.pt
✅ Loaded model global_round_3.pt
✅ Loaded model global_round_4.pt
Loading models from: /Users/yuandouwang/Documents/projects/federated_lab/runs/benign/20250813-223907/models
✅ Loaded model global_round_0.pt
✅ Loaded model global_round_1.pt
✅ Loaded model global_round_2.pt
✅ Loaded model global_round_3.pt
✅ Loaded model global_round_4.pt
✅ Loaded model global_round_5.pt
Loading models from: /Users/yuandouwang/Documents/projects/federated_lab/runs/benign/20250813-224933/models
✅ Loaded model global_round_0.pt
✅ Loaded model global_round_0_base.pt
✅ Loaded model global_round_1.pt
✅ Loaded model global_round_1_base.pt
✅ Loaded model global_round_2.pt
✅ Loaded model global_round_2_base.pt
✅ Loaded model global_round_3.pt
✅ Loaded model global_round_3_base.pt
✅ Loaded model global_round_4

,exp,ts,round,fname,state_dict,model
0,benign,20250813-223511,0,global_round_0.pt,"{'fc.weight': [[tensor(-0.0332), tensor(0.0229...",None
1,benign,20250813-223511,1,global_round_1.pt,"{'fc.weight': [[tensor(-0.0332), tensor(0.0229...",None
2,benign,20250813-223511,2,global_round_2.pt,"{'fc.weight': [[tensor(-0.0332), tensor(0.0229...",None


In [6]:
def extract_model_updates_from_df(df: pd.DataFrame):
    
    updates_map = {}
    for _, row in df.iterrows():
        if "round" not in row:
            continue
        r = int(row["round"])
        if pd.notna(row.get("model_update_cid")):
            nid = int(row["node_id"])
            updates_map.setdefault(r, []).append(
                (nid, row["model_update_cid"], row.get("update_type", "delta"), row.get("acc", None))
            )
    return updates_map

updates_map  = extract_model_updates_from_df(df)

print(updates_map)
# print(globals_map)
ipfs = IPFSSim()
# agg = Aggregator(ipfs, None, [], save_dir=None)



{0: [(0, 'Qm00000001', 'delta', 0.91415), (1, 'Qm00000003', 'delta', 0.945145), (2, 'Qm00000005', 'delta', 0.912869), (3, 'Qm00000007', 'delta', 0.900925), (4, 'Qm00000009', 'delta', 0.924696), (5, 'Qm00000011', 'delta', 0.901916), (6, 'Qm00000013', 'delta', 0.881536), (7, 'Qm00000015', 'delta', 0.907874), (8, 'Qm00000017', 'delta', 0.921027), (9, 'Qm00000019', 'delta', 0.891692), (0, 'Qm00000001', 'delta', 0.913603), (1, 'Qm00000003', 'delta', 0.94516), (2, 'Qm00000005', 'delta', 0.913096), (3, 'Qm00000007', 'delta', 0.899948), (4, 'Qm00000009', 'delta', 0.923776), (5, 'Qm00000011', 'delta', 0.902349), (6, 'Qm00000013', 'delta', 0.881231), (7, 'Qm00000015', 'delta', 0.908131), (8, 'Qm00000017', 'delta', 0.920704), (9, 'Qm00000019', 'delta', 0.892296), (0, 'Qm00000001', 'delta', 0.913213), (1, 'Qm00000003', 'delta', 0.944625), (2, 'Qm00000005', 'delta', 0.912263), (3, 'Qm00000007', 'delta', 0.899782), (4, 'Qm00000009', 'delta', 0.92266), (5, 'Qm00000011', 'delta', 0.901628), (6, 'Qm000

In [ ]:
import re
import os
from pathlib import Path
from typing import Dict, List, Tuple, Optional

import torch
import pandas as pd

# uses your updated eval.py
from flsim.eval import evaluate_reconstructed_batch


# --------- filename parsers ---------

_RX_GLOBAL = re.compile(r"^global_round_(\d+)_base\.(pt|pth)$")
_RX_DELTA  = re.compile(r"^round_(\d+)_node_(\d+)_delta\.(pt|pth)$")


def _index_globals(models_dir: Path) -> Dict[int, dict]:
    """Scan models/ for global_round_{r}.pt -> {r: state_dict}."""
    out: Dict[int, dict] = {}
    if not models_dir.exists():
        return out
    for fname in os.listdir(models_dir):
        m = _RX_GLOBAL.match(fname)
        if not m:
            continue
        r = int(m.group(1))
        fpath = models_dir / fname
        try:
            out[r] = torch.load(fpath, map_location="cpu")
        except Exception as e:
            print(f"⚠️ Failed to load {fpath}: {e}")
    return out


def _index_updates(updates_dir: Path) -> Dict[int, Dict[int, dict]]:
    """
    Scan updates/ for round_{r}_node_{nid}_delta.pt
    -> { r: { nid: delta_state_dict } }.
    """
    out: Dict[int, Dict[int, dict]] = {}
    if not updates_dir.exists():
        return out
    for fname in os.listdir(updates_dir):
        m = _RX_DELTA.match(fname)
        if not m:
            continue
        r   = int(m.group(1))
        nid = int(m.group(2))
        fpath = updates_dir / fname
        try:
            sd = torch.load(fpath, map_location="cpu")
            out.setdefault(r, {})[nid] = sd
        except Exception as e:
            print(f"⚠️ Failed to load {fpath}: {e}")
    return out


# --------- reconstruction + evaluation ---------

def reconstruct_and_eval_one_run(
    run_dir: Path,                      # e.g. .../runs/<exp>/<ts>
    *,
    dataset: str = "mnist",
    model_hint: Optional[str] = "linear-mnist",
    max_workers: int = 4,
) -> pd.DataFrame:
    """
    For a single run folder with:
      models/global_round_{r}.pt
      updates/round_{r}_node_{nid}_delta.pt
    reconstruct base+delta for each (r, nid) and evaluate accuracy.

    Returns a DataFrame with columns:
      ['round','node_id','eval_acc','exp','ts']
    """
    models_dir  = run_dir / "models"
    updates_dir = run_dir / "updates"

    globals_map  = _index_globals(models_dir)           # {r: global_state at r}
    print(f"Found {len(globals_map)} global models in {models_dir}", globals_map)
    updates_map  = _index_updates(updates_dir)          # {r: {nid: delta}}
    print(f"Found {len(updates_map)} rounds with updates in {updates_dir}", updates_map)
    if not globals_map or not updates_map:
        print(f"Nothing to evaluate in {run_dir}")
        return pd.DataFrame()

    # for round r, the base is global at r-1
    rounds = sorted(updates_map.keys())

    rows = []
    for r in rounds:
        base_r_minus_1 = globals_map.get(r)
        if base_r_minus_1 is None:
            # if round 0 has no base, skip or add your own initial_global.pt logic here
            print(f"⚠️ Missing base model for round {r} (need global_round_{r-1}.pt) in {models_dir}")
            continue

        # collect deltas & node order
        nid_list = sorted(updates_map[r].keys())
        deltas   = [updates_map[r][nid] for nid in nid_list]

        # evaluate reconstructed models: base + delta
        accs = evaluate_reconstructed_batch(
            base_state=base_r_minus_1,
            deltas=deltas,
            dataset=dataset,
            model_hint=model_hint,
            max_workers=max_workers,
        )

        for nid, acc in zip(nid_list, accs):
            rows.append({
                "round": r,
                "node_id": nid,
                "eval_acc": float(acc),
                "exp": run_dir.parent.name,
                "ts": run_dir.name,
            })

    return pd.DataFrame(rows)


def reconstruct_and_eval_all_runs(
    runs_root: Path,                    # either .../runs or the project root containing /runs
    *,
    dataset: str = "mnist",
    model_hint: Optional[str] = "linear-mnist",
    max_workers: int = 4,
) -> pd.DataFrame:
    """
    Walk through runs/*/* and evaluate all runs.
    """
    runs_root = runs_root if runs_root.name == "runs" else (runs_root / "runs")
    all_rows = []
    if not runs_root.exists():
        print("No runs folder found:", runs_root)
        return pd.DataFrame()

    for exp_dir in sorted(runs_root.glob("*")):
        if not exp_dir.is_dir(): 
            continue
        for ts_dir in sorted(exp_dir.glob("*")):
            if not ts_dir.is_dir(): 
                continue
            df_run = reconstruct_and_eval_one_run(
                ts_dir,
                dataset=dataset,
                model_hint=model_hint,
                max_workers=max_workers,
            )
            if not df_run.empty:
                all_rows.append(df_run)

    return pd.concat(all_rows, ignore_index=True) if all_rows else pd.DataFrame()


In [8]:
from pathlib import Path

RUNS = Path("/Users/yuandouwang/Documents/projects/federated_lab/runs")

# 1) One run (exp/timestamp folder)
exp = "benign"          # e.g. use df_all["exp"].unique()[0]
ts  = "20250813-224933"  # e.g. use df_all["ts"].unique()[0]
df_one = reconstruct_and_eval_one_run(RUNS / exp / ts, dataset="mnist", model_hint="linear-mnist")
print(df_one.head())

# 2) All runs
# df_all = reconstruct_and_eval_all_runs(RUNS, dataset="mnist", model_hint="linear-mnist")
# print(df_all.groupby("round")["eval_acc"].mean().round(4))


Found 6 global models in /Users/yuandouwang/Documents/projects/federated_lab/runs/benign/20250813-224933/models {1: {'fc.weight': tensor([[ 0.0091,  0.0308, -0.0257,  ...,  0.0206, -0.0344,  0.0173],
        [-0.0268,  0.0029,  0.0045,  ...,  0.0275,  0.0335, -0.0069],
        [-0.0076,  0.0066,  0.0051,  ..., -0.0219,  0.0033, -0.0072],
        ...,
        [ 0.0025,  0.0129,  0.0086,  ..., -0.0150, -0.0057, -0.0106],
        [ 0.0336,  0.0015, -0.0280,  ..., -0.0304, -0.0030, -0.0251],
        [-0.0256,  0.0222, -0.0286,  ...,  0.0112,  0.0251, -0.0348]]), 'fc.bias': tensor([-0.0946,  0.2198, -0.0476, -0.1155,  0.0483,  0.3540, -0.0457,  0.1803,
        -0.3515, -0.0569])}, 5: {'fc.weight': tensor([[ 0.0091,  0.0308, -0.0257,  ...,  0.0206, -0.0344,  0.0173],
        [-0.0268,  0.0029,  0.0045,  ...,  0.0275,  0.0335, -0.0069],
        [-0.0076,  0.0066,  0.0051,  ..., -0.0219,  0.0033, -0.0072],
        ...,
        [ 0.0025,  0.0129,  0.0086,  ..., -0.0150, -0.0057, -0.0106],
     